In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [47]:
train_path = '/kaggle/input/home-data-for-ml-course/train.csv'
test_path = '/kaggle/input/home-data-for-ml-course/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [48]:
# 범주형 컬럼 가져오기
categorical_cols = train_df.select_dtypes(include=['object']).columns
# 자료가 없다고 처리하기
for col in categorical_cols:
    train_df.fillna({col: 'Missing'}, inplace=True)
    test_df.fillna({col: 'Missing'}, inplace=True)

In [49]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))
    test_df[col] = test_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
    label_encoders[col] = le

# 숫자형 컬럼 - 비어있는 값에는 중앙값으로 처리함
for col in train_df.columns:
    if train_df[col].isnull().sum() > 0:
        train_df.fillna({col: train_df[col].median()}, inplace=True)
        test_df.fillna({col: test_df[col].median()}, inplace=True)

In [50]:
X = train_df.drop(columns=['Id', 'SalePrice']) # y값과 훈련에 필요없는 id 값 제거
y = np.log(train_df['SalePrice'])  # 로그 변환

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [52]:
y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print(f"Validation RMSE: {rmse}")

Validation RMSE: 0.14748040495769896


In [53]:
missing_values_in_test = test_df.isnull().sum()
missing_values_in_test = missing_values_in_test[missing_values_in_test > 0]
missing_values_in_test

BsmtFinSF1      1
BsmtFinSF2      1
BsmtUnfSF       1
TotalBsmtSF     1
BsmtFullBath    2
BsmtHalfBath    2
GarageCars      1
GarageArea      1
dtype: int64

In [54]:
# 결측치가 남아있는 열들에 대해 다시 한 번 중앙값으로 채우기
for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
            'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 
           'GarageCars', 'GarageArea']:
    test_df.fillna({col: test_df[col].median()}, inplace=True)

# 다시 결측치 확인
missing_values_in_test = test_df.isnull().sum()
missing_values_in_test[missing_values_in_test > 0]

Series([], dtype: int64)

In [55]:
test_X = test_df.drop(columns=['Id'])
test_predictions = np.exp(rf_model.predict(test_X))  # Exponentiate to get actual prices
print(test_predictions)

[123540.56749799 152508.62266471 177507.11008462 ... 154229.37019329
 115538.81604727 227449.30101352]


In [56]:
submission = pd.DataFrame({'Id': test_df['Id'], 'SalePrice': test_predictions})
submission_path = "submission.csv"
submission.to_csv(submission_path, index=False)
submission.head()

,Id,SalePrice
0,1461,123540.567498
1,1462,152508.622665
2,1463,177507.110085
3,1464,182757.119619
4,1465,196837.887752
